<a href="https://colab.research.google.com/github/Klnishant/LiveNLPClass/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip -q fra-eng.zip -d /content/french-eng

--2024-06-01 05:41:43--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  21.4MB/s    in 0.4s    

2024-06-01 05:41:43 (21.4 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]



In [2]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


In [3]:
batch_s =64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = '/content/french-eng/fra.txt'

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
  input_text, target_text,_ = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


In [5]:
print("Number of input samples:", len(input_texts))
print("NUmber of target samples:", len(target_texts))
print("Unique input characters:", len(input_characters))
print("Unique target characters:", len(target_characters))

Number of input samples: 10000
NUmber of target samples: 10000
Unique input characters: 70
Unique target characters: 91


In [6]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tEn route !\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tWaouh\xa0!\n',
 '\tWah\xa0!\n',
 '\tÀ terre\xa0!\n',
 '\tBaisse-toi\xa0!\n',
 '\tBaissez-vous\xa0!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tCache-toi.\n',
 '\tCachez-vous.\n',
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\t

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [8]:
max_encoder_seq_length

14

In [9]:
max_decoder_seq_length

59

In [10]:
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [12]:
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

In [13]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [14]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 '?': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'D': 22,
 'E': 23,
 'F': 24,
 'G': 25,
 'H': 26,
 'I': 27,
 'J': 28,
 'K': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '\xa0': 69,
 '«': 70,
 '»': 71,
 'À': 72,
 'Ç': 73,
 'É': 74,
 'Ê': 75,
 'à': 76,
 'â': 77,
 'ç': 78,
 'è': 79,
 'é': 80,
 'ê': 81,
 'î': 82,
 'ï': 83,
 'ô': 84,
 'ù': 85,
 'û': 86,
 'œ': 87,
 '\u2009': 88,
 '’': 89,
 '\u202f': 90}

In [15]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [16]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] =1.
    if t>0:
      decoder_target_data[i, t-1, target_token_index[char]] =1.
  decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
  decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [17]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [20]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [21]:
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)

In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data, batch_size=batch_s, epochs=epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 7s 22ms/step - loss: 1.2000 - accuracy: 0.7340 - val_loss: 1.0692 - val_accuracy: 0.7115
Epoch 2/100
125/125 [==============================] - 1s 11ms/step - loss: 0.9276 - accuracy: 0.7496 - val_loss: 0.9519 - val_accuracy: 0.7308
Epoch 3/100
125/125 [==============================] - 1s 10ms/step - loss: 0.8415 - accuracy: 0.7683 - val_loss: 0.8711 - val_accuracy: 0.7549
Epoch 4/100
125/125 [==============================] - 1s 10ms/step - loss: 0.7424 - accuracy: 0.7917 - val_loss: 0.7794 - val_accuracy: 0.7741
Epoch 5/100
125/125 [==============================] - 1s 10ms/step - loss: 0.6740 - accuracy: 0.8064 - val_loss: 0.7189 - val_accuracy: 0.7901
Epoch 6/100
125/125 [==============================] - 1s 10ms/step - loss: 0.6310 - accuracy: 0.8165 - val_loss: 0.6850 - val_accuracy: 0.8001
Epoch 7/100
125/125 [==============================] - 1s 11ms/step - loss: 0.6017 - accuracy: 0.8245 - val_loss: 0.6684 - val_accuracy:

In [24]:
encoder_model = Model(encoder_inputs,encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)

In [25]:
reverse_input_char_index = dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index = dict((i,char) for char,i in target_token_index.items())

In [26]:
reverse_input_char_index

{0: ' ',
 1: '!',
 2: '"',
 3: '$',
 4: '%',
 5: '&',
 6: "'",
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: '2',
 13: '3',
 14: '5',
 15: '7',
 16: '8',
 17: '9',
 18: ':',
 19: '?',
 20: 'A',
 21: 'B',
 22: 'C',
 23: 'D',
 24: 'E',
 25: 'F',
 26: 'G',
 27: 'H',
 28: 'I',
 29: 'J',
 30: 'K',
 31: 'L',
 32: 'M',
 33: 'N',
 34: 'O',
 35: 'P',
 36: 'Q',
 37: 'R',
 38: 'S',
 39: 'T',
 40: 'U',
 41: 'V',
 42: 'W',
 43: 'Y',
 44: 'a',
 45: 'b',
 46: 'c',
 47: 'd',
 48: 'e',
 49: 'f',
 50: 'g',
 51: 'h',
 52: 'i',
 53: 'j',
 54: 'k',
 55: 'l',
 56: 'm',
 57: 'n',
 58: 'o',
 59: 'p',
 60: 'q',
 61: 'r',
 62: 's',
 63: 't',
 64: 'u',
 65: 'v',
 66: 'w',
 67: 'x',
 68: 'y',
 69: 'z'}

In [27]:
reverse_target_char_index

{0: '\t',
 1: '\n',
 2: ' ',
 3: '!',
 4: '%',
 5: '&',
 6: "'",
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: '2',
 13: '3',
 14: '5',
 15: '8',
 16: '9',
 17: ':',
 18: '?',
 19: 'A',
 20: 'B',
 21: 'C',
 22: 'D',
 23: 'E',
 24: 'F',
 25: 'G',
 26: 'H',
 27: 'I',
 28: 'J',
 29: 'K',
 30: 'L',
 31: 'M',
 32: 'N',
 33: 'O',
 34: 'P',
 35: 'Q',
 36: 'R',
 37: 'S',
 38: 'T',
 39: 'U',
 40: 'V',
 41: 'W',
 42: 'Y',
 43: 'a',
 44: 'b',
 45: 'c',
 46: 'd',
 47: 'e',
 48: 'f',
 49: 'g',
 50: 'h',
 51: 'i',
 52: 'j',
 53: 'k',
 54: 'l',
 55: 'm',
 56: 'n',
 57: 'o',
 58: 'p',
 59: 'q',
 60: 'r',
 61: 's',
 62: 't',
 63: 'u',
 64: 'v',
 65: 'w',
 66: 'x',
 67: 'y',
 68: 'z',
 69: '\xa0',
 70: '«',
 71: '»',
 72: 'À',
 73: 'Ç',
 74: 'É',
 75: 'Ê',
 76: 'à',
 77: 'â',
 78: 'ç',
 79: 'è',
 80: 'é',
 81: 'ê',
 82: 'î',
 83: 'ï',
 84: 'ô',
 85: 'ù',
 86: 'û',
 87: 'œ',
 88: '\u2009',
 89: '’',
 90: '\u202f'}

In [34]:
def decoded_seq(input_seq):
  states_value = encoder_model.predict(input_seq)

  target_seq = np.zeros((1,1,num_decoder_tokens))
  target_seq[0,0,target_token_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq]+states_value)
    sampled_token_index = np.argmax(output_tokens[0,-1,:])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char

    if(sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    target_seq = np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,sampled_token_index] = 1.
    states_value = [h,c]
  return decoded_sentence

In [36]:
for seq_index in range(10):
  input_seq = encoder_input_data[seq_index:seq_index+1]
  decoded_sentence = decoded_seq(input_seq)
  print('_')
  print(input_texts[seq_index])
  print('decoded_sentence:', decoded_sentence)


1/1 [==============================] - 0s 20ms/step
_
Go.
decoded_sentence: Contrez.

1/1 [==============================] - 0s 26ms/step
_
Go.
decoded_sentence: Contrez.

1/1 [==============================] - 0s 19ms/step
_
Go.
decoded_sentence: Contrez.

1/1 [==============================] - 0s 19ms/step
_
Go.
decoded_sentence: Contrez.

1/1 [==============================] - 0s 20ms/step
_
Hi.
decoded_sentence: Ealut-il !

1/1 [==============================] - 0s 19ms/step
_
Hi.
decoded_sentence: Ealut-il !

1/1 [==============================] - 0s 23ms/step
_
Run!
decoded_sentence: File !

1/1 [==============================] - 0s 20ms/step
_
Run!
decoded_sentence: File !

1/1 [==============================] - 0s 19ms/step
_
Run!
decoded_sentence: File !

1/1 [==============================] - 0s 21ms/step
_
Run!
decoded_sentence: File !

